In [1]:
# !pip install smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade -q

In [2]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "lm studio"
openai_api_base = "http://localhost:1234/v1" #"http://192.168.1.143:8080/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
# QwQ-32B-Q4_K_M qwq-32b
def llm_quote(model_name="qwen2.5-14b-instruct-1m@q4_k_m", tsystem="你是强大的助手", tprompt="千问强还是deepseek强？"):
    completion = client.chat.completions.create(
        model=model_name, #,"qwen2.5-14b-instruct-1m@q4_k_m" 
        messages=[
            {"role": "system", "content": tsystem},
            {
                "role": "user",
                "content": tprompt,
            },
        ],
    )
    chat_response = completion.choices[0].message.content
    # print("Chat response:\n", chat_response)
    return chat_response

In [3]:
sys_prompt_key_words = "仅输出用户输入里的两个最关键的名词性短语，以空格分开，不要想象，不要引申"
# sys_prompt_quote = 
user_prompt = "详细介绍梅花易数的起卦方法"

In [4]:
# get key words
# tresponse = llm_quote(tsystem=sys_prompt_key_words,tprompt=user_prompt)
# tid = tresponse.find("</think>")
# tanswer = tresponse[tid:].split()
# print(tanswer)
# keyws = tanswer[1:3]
keyws = ['梅花易数','起卦方法']

In [1]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.retrievers import BM25Retriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
# from dotenv import load_dotenv
import os
import shutil

# load_dotenv()

def load_and_process_txts(data_dir: str):
    """Load TXTs from directory and split into chunks."""
    loader = DirectoryLoader(
        data_dir,
        glob="**/*.txt",
        loader_cls=TextLoader, #PyPDFLoader
        loader_kwargs={"encoding":"utf-8"}
    )
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],

    )
    chunks = text_splitter.split_documents(documents)
    
    return chunks

tchuncks = load_and_process_txts("data_yi")
print(len(tchuncks))
# print(tchuncks[141])
# print("  ")
# print(tchuncks[351])

1976


In [6]:
import jieba
def cut_words(text):
    return jieba.lcut(text)
retriever = BM25Retriever.from_documents(tchuncks,preprocess_func=cut_words, k=10)

# trdocs = retriever.invoke("潜龙勿用")
# print(trdocs)

# import jieba.analyse
# tprompt = f"详细解释利见大人"
# # sent = "从本地知识库中找例子详细说明梅花易数预测方法包括：起卦，排挂, 解卦三个步骤"
# tplen = len(tprompt)
# print(tplen)
# ttk = int(tplen / 2) * 2
# print(ttk)
# keyws = jieba.analyse.extract_tags(tprompt, topK=1, withWeight=False, allowPOS=())
# keyws.append(tprompt)
# print(keyws)

tcontext = "相关内容：\n"
for keyw in keyws:
    print("keyw: ", keyw)
    trdocs = retriever.invoke(keyw)
    # print(trdocs)
    tcontext = tcontext.join(tdoc.page_content for tdoc in trdocs)
    print(len(tcontext))
# print(tcontext)
tcontext = tcontext[:125000]
print(len(tcontext))
tsystem = f"你是强大的知识查询助手。本次对话的相关内容如下，请根据相关内容和你的理解回答用户的问题： {tcontext}"
print(len(tsystem))

Building prefix dict from the default dictionary ...
Loading model from cache G:\systemp\jieba.cache
Loading model cost 0.567 seconds.
Prefix dict has been built successfully.


keyw:  梅花易数
4562
keyw:  起卦方法
45514
45514
45559


In [7]:
tresponse = llm_quote(tsystem=tsystem,tprompt=user_prompt)
print(tresponse)

APITimeoutError: Request timed out.